# 仮説と推定
分析は、データとそれを生成するプロセスに関する仮説をたて、検定を行う作業が含まれます

## 統計的仮説検定
ある仮説が真である可能性が高いか否かを示さなければならない場合があります。
データに関する統計量に言い換えられる、ある種の主張が仮説です。
統計量とは、様々な仮定がどの程度確からしいかを提示できます。

古典的な設定では、帰無仮説H0が基本的な立ち位置を表し、対立仮説H1と比較されます。
統計量を用いて、帰無仮説を棄却できるかを決定します。

## コイン投げの例
コインに歪みがあるかを確認したい。
コインを投げて表がでる確率をpとした場合、コインが歪みがないことを示す帰無仮説は、p=0.5になる。
検定を用いて比較する。

検定では、コインをn回投げて、表が出た回数Xを数えます。
コイン投げはベルヌーイ試行にあたり、Xは二項分布 binomial(n,p)の確率変数になる。


In [15]:
import math
def normal_approximation(n,p):
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# 正規分布の累積分布関数
def normal_cdf(x, mu=0, sugma=1):
    return (1 + math.erf((x -mu) / math.sqrt(2) / sigma)) / 2


# 特定の確率となる値を見つけるために、nourmal_cdfの逆関数必要
def inverse_nomal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    """二分探索を用いて、逆関数の近似値を計算"""
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_nomal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0
    hi_z, hi_p = 10.0, 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid < p:
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_z, mid_p
        else:
            break
            
    return mid_z
    

コイン投げの回数をn=1000とします。
コインの歪みがないという仮説が真であるなら、Xは平均500で分散15.8で近似できます

In [7]:
mu_0, sigma_0 = normal_approximation(1000,0.5)
print(mu_0, sigma_0)

500.0 15.811388300841896


## ベイズの推定

検定を用いて、2郡の差をみた場合「帰無仮説が正しいときには、極端に違いが出る確認がXX％しかない」という結論が出ます。
これとは、別に、未知のパラメーターを確率変数として扱う推定の手法があります。
パラメーターの事前分布に対して、観測データとべーずの定理を用いて、パラメーターの事後分布を求めます。
検定結果の確率を使う代わりにパラメーター自身を用いて、確率的な判断を行う方法がベイズ推定です。


β分布は、0から1の間の様々な値を取りうる分布です。ベイズでは都合が良いために、頻繁に利用します

In [18]:
import math
def B(alpha, beta):
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x, alpha, beta):
    if x < 0 or x > 1:
        return 0
    return x ** (alpha -1) * (1 - x) ** (beta -1) / B(alpha, beta)

一般的にこのβ分布は、以下の重みを中心にします。
$${\frac{alpha}{(alpha + beta)}}$$

alphaとbetaが大きい場合、分散は狭くなります。

### β分布における、aplphaとbetaの関係
alphaとbetaが共に1である場合、一様分布になります。
alphaがbetaより大きい場合、1の付近に重みが偏ります。
betaがalphaより大きい場合、0の付近に重みが偏ります。
alphaとbetaが1以外の同一の値を持つ場合、0.5付近に重みが偏ります。


### ベイズ推定のモチベーション
コイン投げを何度も行い、表が出た回数hと裏がでた回数tを観察します。
ベイズ推定では、事前分布を仮定する必要があります。

#### 事前分布の決め方の例として
* コインに偏りがない：alphaとbetaは、1として一様分布。
* 偏りがあり55％で表が出る：alpha=55, beta=45としても良いです。

#### 事後分布はどうなるか？
事前分布を決めたうえで、コイン投げを実施し、表がでた回数をh、裏が出た回数をtとして観察します。
観察結果を事後分布に当てはめ、事後分布を作成します。
事後分布も事前分布と同じく、β分布になります。事後分布は、観察結果が反映され以下のようになります。

alpha + h, alpha + t

これらを繰り返すことで真の確率分布が見えてきます。
事前分布を適当に設定しても、事後の観察結果により、正しい分布に近寄って来ます。